In [236]:
!{sys.executable} -m pip install python-craigslist

/bin/sh: {sys.executable}: command not found
/bin/sh: {sys.executable}: command not found


In [93]:
import sys
!{sys.executable} -m pip install python-craigslist
!{sys.executable} -m pip install python-crontab

thinc 6.11.2 requires hypothesis<3,>=2, which is not installed.
thinc 6.11.2 requires pathlib<2.0.0,>=1.0.0, which is not installed.
spacy 2.0.11 requires pathlib, which is not installed.
smart-open 1.6.0 requires bz2file, which is not installed.
msgpack-numpy 0.4.3 requires msgpack>=0.3.0, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
thinc 6.11.2 has requirement msgpack-numpy==0.4.1, but you'll have msgpack-numpy 0.4.3 which is incompatible.
thinc 6.11.2 has requirement msgpack-python==0.5.4, but you'll have msgpack-python 0.5.6 which is incompatible.
spacy 2.0.11 has requirement regex==2017.4.5, but you'll have regex 2018.6.21 which is incompatible.
spacy 2.0.11 has requirement thinc<6.11.0,>=6.10.1, but you'll have thinc 6.11.2 which is incompatible.
ipywidgets 7.1.0 has requirement widgetsnbextension~=3.1.0, but you'll have widgetsnbextension 3.0.2 which is incompatible.
You are using pip version 10.0.1, however version 18.0 is available.
You

In [1]:
from craigslist import CraigslistHousing
from math import radians, cos, sin, asin, sqrt
import pandas as pd
import csv
import numpy as np
import argparse
import json
import pprint
import requests
import sys
import urllib
from crontab import CronTab

In [2]:
from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode

In [3]:
API_KEY= '3nU3phN37fdNEA15I_fXlqa_E_zGaHQlaKho1jMgpKWxShPokBFLiSSHes_GTPYMr93WFF-hpFvVThpoFkSTHBKXUxrSADaHBqeePHflHk-fRWlUa-NleO-pxOWoW3Yx' 
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 3

In [4]:
app_id = 'oZHdFiJCLnokObkQ0RPJpw'
app_secret = API_KEY

In [5]:
def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

In [6]:
def search(api_key, category, latitude, longitude):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'latitude': latitude,
        'longitude': longitude,
        'limit': SEARCH_LIMIT,
        'categories': category,
        'radius': 800,
        'price':[1,2]
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)


In [7]:
def get_business(api_key, business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)


In [8]:
def query_api(term, location):
    """Queries the API by the input values from the user.
    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(API_KEY, term, location)

    businesses = response.get('businesses')

    if not businesses:
        print(u'No businesses for {0} in {1} found.'.format(term, location))
        return

    business_id = businesses[0]['id']

    print(u'{0} businesses found, querying business info ' \
        'for the top result "{1}" ...'.format(
            len(businesses), business_id))
    response = get_business(API_KEY, business_id)

    print(u'Result for business "{0}" found:'.format(business_id))
    pprint.pprint(response, indent=2)

In [116]:
cl = CraigslistHousing(site='newyork', category='aap',
                         filters={'max_price': 9000, 'min_price': 1000})

In [117]:
results = cl.get_results(sort_by='newest', geotagged=True, limit=1000)
results = [result for result in results]

In [118]:
express_trains = {
    "72nd Street":(40.76871175277928,-73.95845890045166),
    "96th Street":(40.794220933640126,-73.97214889526367),
    "Rockefeller Center":(40.7586678722452,-73.98133277893066),
    "W 4th St":(40.7342315,-73.99880200000001),
    "125th St": (40.811568076586966,-73.95206451416016),
    "Canal St":(40.7182002,-73.99357359999999),
    "Grand St":(40.7182002,-73.99357359999999),
    "34th St":(40.750568,-73.99351899999999),
    "14th St":(40.7355816,-73.99219579999999),
    "Times Square": (40.7560445,-73.98778329999999),
    "Chambers Street": (40.76871175277928,-73.95845890045166),
    "57th Street": (37.790588199999995,-122.40038539999998)
}

In [119]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [120]:
geocode = [result["geotag"] for result in results]
 

In [121]:
areas = []
for result in results:
    if result["area"] != None:
        areas.append(result["area"])
    else:
        areas.append(242)

In [122]:
bedrooms = []
for result in results:
    if result["bedrooms"] != None:
        bedrooms.append(result["bedrooms"])
    else:
        bedrooms.append(0)

In [123]:
latitudes = []
longitudes = []
for geo in geocode:
    if geo != None:
        latitudes.append(geo[0])
        longitudes.append(geo[1])
    else:
        latitudes.append(0)
        longitudes.append(0)

In [124]:
near_train_results = []
near_pizza = []
near_bagels = []
near_convenience_store = []
near_cocktail_bars = []
near_food_trucks = []

for lat, long in zip(latitudes, longitudes):
    near_train = False
    for express_train in express_trains:
        if haversine(long, lat, express_trains[express_train][1], express_trains[express_train][0]) < 1:
            near_train = True
    if near_train == True:
        near_train_results.append(1)
    else:
        near_train_results.append(0)
    near_pizza.append(len(search(API_KEY, "pizza", lat, long)))
    near_bagels.append(len(search(API_KEY, "bagels", lat, long)))
    near_convenience_store.append(len(search(API_KEY, "convenience", lat, long)))
    near_cocktail_bars.append(len(search(API_KEY, "cocktailbars", lat, long)))
    near_food_trucks.append(len(search(API_KEY, "foodtrucks", lat, long)))

Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying h

Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying h

Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying h

Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying h

Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying h

...

Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying h

Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying h

Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying h

Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying h

Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying https://api.yelp.com/v3/businesses/search ...
Querying h

In [125]:
df = pd.DataFrame(results)
df.dtypes

area         object
bedrooms     object
datetime     object
geotag       object
has_image      bool
has_map        bool
id           object
name         object
price        object
repost_of    object
url          object
where        object
dtype: object

In [126]:
df["Latitude"] = latitudes

In [127]:
df["Longitude"] = longitudes

In [128]:
df["area"] = areas

In [129]:
df["bedrooms"] = bedrooms

In [130]:
df["Near_Express_Train"] = near_train_results

In [131]:
df["near_pizza"] = near_pizza
df["near_bagels"] = near_bagels
df["near_convenience_store"] = near_convenience_store
df["near_cocktail_bars"] = near_cocktail_bars
df["near_food_trucks"] = near_food_trucks

Convert housing prices into a numerical set of values.

In [132]:
df["price"] = df["price"].apply(lambda x: x.replace('$','')).apply(lambda x: x.replace(',','')).astype(float)

In [133]:
df["area"] = df["area"].astype(str)

In [134]:
df

,area,bedrooms,datetime,geotag,has_image,has_map,id,name,price,repost_of,url,where,Latitude,Longitude,Near_Express_Train,near_pizza,near_bagels,near_convenience_store,near_cocktail_bars,near_food_trucks
0,777ft2,1,2018-10-02 13:39,"(30.394563, -95.654536)",True,True,6700053094,Live in Luxury on Lake Conroe!,1060.0,6598988092,https://houston.craigslist.org/apa/d/live-in-l...,Harbor Shores,30.394563,-95.654536,0,1,1,1,1,1
1,922ft2,2,2018-10-02 13:36,"(29.971387, -95.546334)",True,True,6713146797,Great low rates on spacious floor plans!,1039.0,None,https://houston.craigslist.org/apa/d/great-low...,None,29.971387,-95.546334,0,1,1,1,1,1
2,1005ft2,3,2018-10-02 13:36,"(30.347806, -95.076885)",True,True,6713146413,Nice Place To Call Home!!Come in and Ask about...,1079.0,5204936318,https://houston.craigslist.org/apa/d/nice-plac...,"Cleveland,Tx",30.347806,-95.076885,0,1,1,1,1,1
3,242,3,2018-10-02 13:35,"(30.2257, -95.4924)",True,True,6713146089,Everything You Need To Live An Active And Sati...,1555.0,None,https://houston.craigslist.org/apa/d/everythin...,Woodlands/ Conroe South,30.225700,-95.492400,0,1,1,1,1,1
4,814ft2,1,2018-10-02 13:34,"(29.874151, -95.674365)",True,True,6713139462,"Private Patio or Balcony, High-End Crown Molding",1077.0,None,https://houston.craigslist.org/apa/d/private-p...,Eldridge West Oaks,29.874151,-95.674365,0,1,1,1,1,1
5,1341ft2,0,2018-10-02 13:34,"(30.2257, -95.4924)",False,True,6713145163,"Mulit-Media Outlets in Every Room, Group Exerc...",1700.0,None,https://houston.craigslist.org/apa/d/mulit-med...,Conroe,30.225700,-95.492400,0,1,1,1,1,1
6,1005ft2,3,2018-10-02 13:34,"(30.347787, -95.077572)",True,True,6713144505,Free Rent on Selected Units-Stop by today-Offe...,1079.0,5203514537,https://houston.craigslist.org/apa/d/free-rent...,"Cleveland,Tx",30.347787,-95.077572,0,1,1,1,1,1
7,580ft2,1,2018-10-02 13:33,"(29.801563, -95.446682)",True,True,6709249832,"☆ Beautiful Houston apartments, Gourmet Prep K...",1170.0,None,https://houston.craigslist.org/apa/d/beautiful...,None,29.801563,-95.446682,0,1,1,1,1,1
8,242,2,2018-10-02 13:33,"(29.919, -95.5569)",True,True,6713133618,Spacious And Modern Apartment Living In Champions,1105.0,None,https://houston.craigslist.org/apa/d/spacious-...,Willowbrook/ Champions/ Ella,29.919000,-95.556900,0,1,1,1,1,1
9,800ft2,2,2018-10-02 13:32,"(29.737998, -95.452137)",True,True,6705048648,Mid-Century Modern ~ 2 bed ~ hardwoods ~ centr...,1295.0,6275851040,https://houston.craigslist.org/apa/d/mid-centu...,Highland Village / Afton Oaks,29.737998,-95.452137,0,1,1,1,1,1


In [135]:
df["Interested"] = np.where((df["price"] > 1000) & (df["price"] < 2000) & (df["Near_Express_Train"] == 1),1,0)



In [136]:
df["area"] = df["area"].apply(lambda x: x.replace('ft2','')).astype(float)

In [137]:
df

,area,bedrooms,datetime,geotag,has_image,has_map,id,name,price,repost_of,...,where,Latitude,Longitude,Near_Express_Train,near_pizza,near_bagels,near_convenience_store,near_cocktail_bars,near_food_trucks,Interested
0,777.0,1,2018-10-02 13:39,"(30.394563, -95.654536)",True,True,6700053094,Live in Luxury on Lake Conroe!,1060.0,6598988092,...,Harbor Shores,30.394563,-95.654536,0,1,1,1,1,1,0
1,922.0,2,2018-10-02 13:36,"(29.971387, -95.546334)",True,True,6713146797,Great low rates on spacious floor plans!,1039.0,None,...,None,29.971387,-95.546334,0,1,1,1,1,1,0
2,1005.0,3,2018-10-02 13:36,"(30.347806, -95.076885)",True,True,6713146413,Nice Place To Call Home!!Come in and Ask about...,1079.0,5204936318,...,"Cleveland,Tx",30.347806,-95.076885,0,1,1,1,1,1,0
3,242.0,3,2018-10-02 13:35,"(30.2257, -95.4924)",True,True,6713146089,Everything You Need To Live An Active And Sati...,1555.0,None,...,Woodlands/ Conroe South,30.225700,-95.492400,0,1,1,1,1,1,0
4,814.0,1,2018-10-02 13:34,"(29.874151, -95.674365)",True,True,6713139462,"Private Patio or Balcony, High-End Crown Molding",1077.0,None,...,Eldridge West Oaks,29.874151,-95.674365,0,1,1,1,1,1,0
5,1341.0,0,2018-10-02 13:34,"(30.2257, -95.4924)",False,True,6713145163,"Mulit-Media Outlets in Every Room, Group Exerc...",1700.0,None,...,Conroe,30.225700,-95.492400,0,1,1,1,1,1,0
6,1005.0,3,2018-10-02 13:34,"(30.347787, -95.077572)",True,True,6713144505,Free Rent on Selected Units-Stop by today-Offe...,1079.0,5203514537,...,"Cleveland,Tx",30.347787,-95.077572,0,1,1,1,1,1,0
7,580.0,1,2018-10-02 13:33,"(29.801563, -95.446682)",True,True,6709249832,"☆ Beautiful Houston apartments, Gourmet Prep K...",1170.0,None,...,None,29.801563,-95.446682,0,1,1,1,1,1,0
8,242.0,2,2018-10-02 13:33,"(29.919, -95.5569)",True,True,6713133618,Spacious And Modern Apartment Living In Champions,1105.0,None,...,Willowbrook/ Champions/ Ella,29.919000,-95.556900,0,1,1,1,1,1,0
9,800.0,2,2018-10-02 13:32,"(29.737998, -95.452137)",True,True,6705048648,Mid-Century Modern ~ 2 bed ~ hardwoods ~ centr...,1295.0,6275851040,...,Highland Village / Afton Oaks,29.737998,-95.452137,0,1,1,1,1,1,0


In [138]:
# with open('housing.csv', 'wb') as csvfile:
#     writer = csv.writer(csvfile, delimiter=' ',
#                             quotechar='|', quoting=csv.QUOTE_MINIMAL)

In [139]:
# df.to_csv('housing.csv')

In [140]:
with open('housing.csv', 'a') as f:
    df.to_csv(f, header=False)